# 전처리

In [ ]:
# import
import pandas as pd
df = pd.read_csv('events.csv')

# ID 정보 문자열 변환

df['user_id'] = df['user_id'].astype(str)
df['category_id'] = df['category_id'].astype(str)
df['product_id'] = df['product_id'].astype(str)

# 날짜 구분 컬럼(년/월/일)

df['event_time'] = pd.to_datetime(df['event_time'])
df['year'], df['month'], df['day'], df['hour'] = df['event_time'].dt.year,  df['event_time'].dt.month,  df['event_time'].dt.day, df['event_time'].dt.hour
df[['main_category','sub_category','sub_sub_category']] = df['category_code'].str.split('.', n=2,expand=True)

# user_session별 지속시간

session_duration = df.groupby('user_session')['event_time'].agg(['min', 'max'])
session_duration['duration'] = session_duration['max'] - session_duration['min']

# user_session별 발생 이벤트
session_events = df.groupby('user_session')['event_type'].value_counts().unstack().fillna(0)

In [ ]:
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,year,month,day,hour,main_category,sub_category,sub_sub_category
0,2020-09-24 11:57:06+00:00,view,1996170,2144415922528452715,electronics.telephone,NaN,31.90,1515915625519388267,LJuJVLEjPT,2020,9,24,11,electronics,telephone,None
1,2020-09-24 11:57:26+00:00,view,139905,2144415926932472027,computers.components.cooler,zalman,17.16,1515915625519380411,tdicluNnRY,2020,9,24,11,computers,components,cooler
2,2020-09-24 11:57:27+00:00,view,215454,2144415927158964449,NaN,NaN,9.81,1515915625513238515,4TMArHtXQy,2020,9,24,11,NaN,NaN,NaN
3,2020-09-24 11:57:33+00:00,view,635807,2144415923107266682,computers.peripherals.printer,pantum,113.81,1515915625519014356,aGFYrNgC08,2020,9,24,11,computers,peripherals,printer
4,2020-09-24 11:57:36+00:00,view,3658723,2144415921169498184,NaN,cameronsino,15.87,1515915625510743344,aa4mmk0kwQ,2020,9,24,11,NaN,NaN,NaN


In [ ]:
df.to_csv('events_bu.csv', index=False)

# 데이터 분석 프로젝트 2차

In [ ]:
import pandas as pd
df = pd.read_csv('events_bu.csv')

# Acqusition - 유입
---

Unique Visitors - 유일 방문자 수

- DAU - 일자별 방문자 수
- WAU - 주차별 방문자 수
- MAU - 월별 방문자 수

In [ ]:
# event_data 그룹화
df['event_time'] = pd.to_datetime(df['event_time'])
df['event_date'] = pd.to_datetime(df['event_time']).dt.date
gr=df.groupby('event_date')

In [ ]:
# DAU
DAU = gr['user_id'].nunique().reset_index()
DAU.columns = ['date', 'unique_visitors']
DAU

,date,unique_visitors
0,2020-09-24,1358
1,2020-09-25,2451
2,2020-09-26,2030
3,2020-09-27,2187
4,2020-09-28,2732
...,...,...
153,2021-02-24,2904
154,2021-02-25,2784
155,2021-02-26,2960
156,2021-02-27,2578


In [1]:
# WAU
df['week'] = df['event_time'].dt.isocalendar().week
WAU = df.groupby(['year', 'week'])['user_id'].nunique()
WAU.columns = ['year','week','unique_visitors']
print(WAU)

NameError: name 'df' is not defined

In [ ]:
# MAU
MAU = df.groupby('month')['user_id'].nunique()
MAU.columns = ['month','unique_visitors']
MAU

month
1     81256
2     74606
9     15334
10    84216
11    92600
12    72137
Name: user_id, dtype: int64

### NRU(Newly Registered Users, 신규 등록 사용자)
- 신규 유저임을 알 수 있는 칼럼이 x
- 처음 활동한 시간이 신규 유저라는 가정하에 진행

In [ ]:
# 처음 활동한 사람이 신규 유저라는 가정
# 유저별 처음으로 접속한 날짜 찾기
first = df.groupby('user_id')['event_time'].min()
first = first.reset_index()

# 처음으로 접속한 날짜를 그룹화 해서 일자별 신규 등록 사용자 찾기
daily_nru = first.groupby(first['event_time'].dt.date)['user_id'].nunique()
daily_nru.columns = ['date', 'nru']
daily_nru

event_time
2020-09-24    1358
2020-09-25    2383
2020-09-26    1907
2020-09-27    2043
2020-09-28    2532
              ... 
2021-02-24    2467
2021-02-25    2343
2021-02-26    2523
2021-02-27    2213
2021-02-28    2374
Name: user_id, Length: 158, dtype: int64

### TTS
- 사용자가 처음 방문한 시간
- 사용자가 구매를 완료한 시간

In [ ]:
# 첫 방문 시간 구하기
first = df.groupby('user_id')['event_time'].min().reset_index()
first.columns = ['user_id', 'first_activity']

# 구매 시간 구하기
purchase_data = df[df['event_type'] == 'purchase']

first_purchase = purchase_data.groupby('user_id')['event_time'].min().reset_index()
first_purchase.columns = ['user_id', 'first_purchase_activity']

tts_data = pd.merge(first, first_purchase, on='user_id', how='inner')

tts_data['TTS'] = (tts_data['first_purchase_activity'] - tts_data['first_activity']).dt.total_seconds() / 3600
#
tts_data.head()

,user_id,first_activity,first_purchase_activity,TTS
0,1515915625353286099,2020-10-03 11:20:33+00:00,2020-10-03 11:23:44+00:00,0.053056
1,1515915625353457259,2020-09-29 05:51:33+00:00,2020-09-29 05:52:55+00:00,0.022778
2,1515915625353534622,2020-10-06 08:29:35+00:00,2020-10-06 10:01:55+00:00,1.538889
3,1515915625353561691,2020-11-24 17:34:42+00:00,2020-11-24 17:37:22+00:00,0.044444
4,1515915625353900095,2020-09-27 14:07:36+00:00,2020-10-15 05:09:35+00:00,423.033056


# Revenue - 수익
---

### 총 매출 (Total Revanue)

In [ ]:
# event_type 이 purchase인것만 추출
purchase_data = df[df['event_type'] == 'purchase']

total_revenue = purchase_data['price'].sum()
print(f"Total Revanue: {total_revenue}")

Total Revanue: 5125395.62


### 구매 전환율 (Puchase Conversion Rate)
- 방문자 중 실제 구매를 한 사용자의 비율
- 총 방문자의 수
- 구매를 한 방문자의 수
- 구매를 한 방문자의 수 / 총 방문자의 수

In [ ]:
# 총 방문자의 수 구하기
total_visit = df['user_id'].nunique()

# 구매자 수 구하기
purchase_val = purchase_data['user_id'].nunique()

# 구매 전환율
purchase_conversion_rate = purchase_val / total_visit
purchase_conversion_rate = round(purchase_conversion_rate, 3)

print(f"Total Visit: {total_visit}")
print(f"Total Revanue: {total_revenue}")
print(f"Purchase Conversion Rate: {purchase_conversion_rate}")

Total Visit: 407283
Total Revanue: 5125395.62
Purchase Conversion Rate: 0.052


### 고객당 평균 매출 (Average Revenue per User, ARPU)
- 총 매출 / 총 사용자 수

In [ ]:
# 총 매출 구하기
purchase_data = df[df['event_type'] == 'purchase']
total_revenue = purchase_data['price'].sum()
# 총 사용자 수 구하기
total_user = df['user_id'].nunique()

# 고객당 평균 매출
ARPU = total_revenue / total_user
print(f"Total Revanue: {total_revenue}")
print(f"Total User: {total_user}")
print(f"ARPU('$'): {ARPU}")

Total Revanue: 5125395.62
Total User: 407283
ARPU('$'): 12.584359327543748


### 세션당 평균 매출 (Average Revenue per Session)
- 총 매출 / 세션 수

In [ ]:
# 총 매출
purchase_data = df[df['event_type'] == 'purchase']
total_revenue = purchase_data['price'].sum()

# 세션 수
total_session = df['user_session'].nunique()

# 세션당 평균 매출 (Average Revenue per Session)
ARPS = total_revenue / total_session

print(f"Total Session: {total_session}")
print(f"Total Revanue: {total_revenue}")
print(f"ARPS('$'): {total_user}")

Total Session: 490398
Total Revanue: 5125395.62
ARPS('$'): 407283


#### 가장 많이 구매한 카테고리 아이템
- 각 카테고리별 구매 이벤트 수를 계산해야함

In [ ]:
# 구매 데이터
purchase_data = df[df['event_type'] == 'purchase']

category_purchase_cnt = purchase_data['main_category'].value_counts().reset_index()
category_purchase_cnt.columns = ['category_name', 'purchase_cnt']
category_purchase_cnt

top_category = category_purchase_cnt.iloc[0]
top_category

category_name    computers
purchase_cnt         16870
Name: 0, dtype: object

In [ ]:
top_category_data = df[df['main_category'] == top_category.iloc[0]]
top_category_data['product_id'].value_counts().reset_index()

,product_id,count
0,1821813,14563
1,4099645,7089
2,3791351,6907
3,809948,4290
4,1830099,4201
...,...,...
11848,582214,1
11849,609901,1
11850,4005165,1
11851,3791137,1


### 주문 빈도 - Order Frequency
- 고객이 반복해서 주문하는 빈도
- 고객별 총 주문 수
- 이를 특정 기간(월, 일, 연간) 동안의 고객수로 나눔

In [ ]:
# 주문 데이터 추출
purchase_data = df[df['event_type'] == 'purchase']

user_order_cnt = purchase_data.groupby('user_id').size().reset_index()
user_order_cnt.columns = ['user_id', 'order_cnt']

total_purchase = user_order_cnt['user_id'].nunique()

# 평균을 구하면 됨
average_order_frequency = user_order_cnt['order_cnt'].mean()
average_order_frequency

1.7530041306796846

In [ ]:
# 월간 주문 빈도
purchase_data = df[(df['event_type'] == 'purchase')]
monthly_order_cnt = purchase_data.groupby(['month', 'user_id']).size().reset_index(name='order_cnt')
# 월 별로 주문 수의 평균 구하기
monthly_order_prequency = monthly_order_cnt.groupby('month')['order_cnt'].mean()
monthly_order_prequency

month
1     1.742821
2     1.760654
9     1.606061
10    1.653319
11    1.750000
12    1.688033
Name: order_cnt, dtype: float64

In [ ]:
# 일자별 주문 빈도

purchase_data = df[df['event_type'] == 'purchase']
daily_order_cnt = purchase_data.groupby(['event_date','user_id']).size().reset_index(name='order_cnt')
# 일자별로 주문 수의 평균 구하기
daily_order_frequency = daily_order_cnt.groupby('event_date')['order_cnt'].mean().reset_index()
daily_order_frequency

,event_date,order_cnt
0,2020-09-24,1.500000
1,2020-09-25,1.419048
2,2020-09-26,1.630137
3,2020-09-27,1.571429
4,2020-09-28,1.694915
...,...,...
153,2021-02-24,1.697674
154,2021-02-25,1.485030
155,2021-02-26,1.751592
156,2021-02-27,1.581560


### 평균 주문 금액 - (Average Order Value, AOV)
- 총 판매 금액 / 총 주문 수

In [ ]:
# 총 판매 가격
total_revenue = purchase_data['price'].sum()

# 총 주문 수 - 행의 수
total_orders = purchase_data.shape[0]

AOV = total_revenue / total_orders
print("Average Order Value (AOV): $", AOV)

Average Order Value (AOV): $ 137.24081882932578
